In [ ]:
# Libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor

# Loading the preprocessed data to aid in creating visualizations
data_full = pd.read_csv('data/curated/final_data_full.csv')
data_num = pd.read_csv('data/curated/final_data_num.csv') 

1. Feature importance graph

In [2]:
# Selecting the features that have an impact on rental prices
X = data_full.drop('price', axis=1) 
y = data_full['price'].values  

# Training the random forest model to identify which features contribute the most to the prediction
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X, y)

# Deriveingfeature importance scores from the Random Forest model
importance_scores = rf_model.feature_importances_

# Creating a DataFrame to rank features by importance and to store the feature names and their corresponding importance scores
feature_importance = pd.DataFrame({'Feature': X.columns, 'Importance': importance_scores})

# Sorting the DataFrame in descending order of importance, so that the most important features appear first
feature_importance = feature_importance.sort_values('Importance', ascending=False)

# Plotting the feature importance graph
plt.figure(figsize=(8, 6))
bars = plt.bar(feature_importance['Feature'], feature_importance['Importance'], color='#FF9999')

# Adding annotations to each bar to display the importance value
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height / 2, f'{height:.2f}', ha='center', va='bottom', color='black')

# Setting titles and labels
plt.title('Feature Importance Graph (Predicting Rental Price)')
plt.xlabel('Feature')
plt.ylabel('Importance')
plt.xticks(rotation=45)

# Adjusting layout to prevent overlap
plt.tight_layout()

# Show the plot to display the relative importance of each feature used in the random forest model
plt.show()

2. Correlation heatmap between features

In [ ]:
# Calculating the correlation matrix
corr_matrix = data_full.corr()

#Plotting the heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt='.2f')
plt.title('Correlation Heatmap Between Features')
plt.show()

3. Top 10 suburbs with highest predicted growth rate

In [4]:
# Selecting forecasted rental price as the main feature that will be used to predict growth rate
# Selecting number of bedrooms, bathrooms and distance to melbourne central as secondary features to predict growth rate
X = data_full[['price', 'bedrooms', 'bathrooms', 'distance_to_melbourne_central']]  
y = data_full['growth_rate']  

# Splitting the data into training data and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training the random forest regressor model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predicting growth rates for the test set
y_pred = rf_model.predict(X_test)

# Coming up with the feature importance 
importance_scores = rf_model.feature_importances_
feature_importance = pd.DataFrame({'Feature': X.columns, 'Importance': importance_scores})
feature_importance.sort_values(by='Importance', ascending=False, inplace=True)

# Calculating feature importance to see which is the main feature that contributes to the prediction of growth rate
plt.figure(figsize=(8, 6))
bars = plt.bar(feature_importance['Feature'], feature_importance['Importance'], color='#FF9999')
plt.title('Feature Importance Graph')
plt.xlabel('Feature')
plt.ylabel('Importance')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Predicting growth rates for all suburbs using the trained model
data_full['predicted_growth_rate'] = rf_model.predict(data_full[['price', 'bedrooms', 'bathrooms', 'distance_to_melbourne_central']])

# Finding the top 10 suburbs with the highest predicted growth rates
top_10_suburbs = data_full[['suburb', 'predicted_growth_rate']].sort_values(by='predicted_growth_rate', ascending=False).head(10)

# Visualizing the top 10 suburbs with the highest predicted growth rate
plt.figure(figsize=(10, 6))
bars = plt.bar(top_10_suburbs['suburb'], top_10_suburbs['predicted_growth_rate'], color=['#FF9999', '#66B2FF', '#99FF99', '#FFCC99', '#FF9966'])
plt.title('Top 10 Suburbs with Highest Predicted Growth Rate (Based on Rental Price)')
plt.xlabel('Suburb')
plt.ylabel('Predicted Growth Rate (%)')
plt.xticks(rotation=45)

for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2.0, height, f'{height:.2f}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()

4. Projected rental growth over 5 years

In [6]:
# Define the initial rental prices for the top 10 suburbs with the highest predicted growth rate
initial_rental_prices = data_full.set_index('suburb')['price'].loc[suburbs].to_dict()

# Defining the predicted growth rates
predicted_growth_rates = data_full.set_index('suburb')['predicted_growth_rate'].loc[suburbs].to_dict()

# Projecting rental prices over the next 5 years in those suburbs
projected_prices = {}
years = [0, 1, 2, 3, 4, 5]

# Calculating the projected price for each suburb
for suburb in suburbs:
    initial_price = initial_rental_prices[suburb]
    growth_rate = predicted_growth_rates[suburb] / 100  # Convert growth rate from percentage
    projected_prices[suburb] = [initial_price * (1 + growth_rate) ** year for year in years]

# Plotting the projected rental price growth over the next 5 years
plt.figure(figsize=(10, 6))
for suburb, prices in projected_prices.items():
    plt.plot(years, prices, marker='o', label=suburb)

plt.title('Projected Rental Price Growth Over 5 Years (Based on Predicted Growth Rates)')
plt.xlabel('Year')
plt.ylabel('Projected Rental Price ($)')
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()

5. Median rental prices of top 7 liveable and affordable suburbs

In [8]:
# Grouping by suburb to calculate median rental prices
median_prices = data_full.groupby('suburb')['price'].median()

# Sorting the median prices in ascending order to find the top 7 suburbs with the lowest median rental prices
median_prices = median_prices.sort_values(ascending=True)

# Setting up the plot
plt.figure(figsize=(10, 6))  

# Plotting the median rental prices for the top 7 affordable suburbs
bars = plt.bar(median_prices.index[:7], median_prices.values[:7], color='#FF6666')  # Setting the color for the bars

# Adding titles and labels
plt.title('Median Rental Prices of Top 7 Liveable and Affordable Suburbs')
plt.xlabel('Suburb')
plt.ylabel('Median Rental Price ($)')

# Rotating the x-axis labels for better readability
plt.xticks(rotation=45)

# Adding data labels (text) on the bars to display the exact median price value for each suburb
for bar in bars:
    height = bar.get_height() 
    plt.text(bar.get_x() + bar.get_width() / 2.0, height / 2, f'{height:.2f}', ha='center', va='center', color='black')

# Final adjustments to the layout
plt.tight_layout()

# Step 9: Displaying the plot as a bar chart with the calculated median rental prices
plt.show()

6. Liveability scores of top 7 affordable suburbs

In [ ]:
# Calculate liveability score for each suburb based on the number of schools, parks, shopping centers, and stations
# Liveability score will be considered as the sum of nearby amenities
data_liveability['liveability_score'] = (
    data_liveability['school_count'] +  
    data_liveability['park_count'] +  
    data_liveability['shopping_center_count'] +  
    data_liveability['station_count']  
)

# Grouping by suburb to calculate total liveability score per suburb and sort in descending order
liveability_scores = data_liveability.groupby('suburb')['liveability_score'].sum()

# Sorting liveability scores to get the top 7 affordable suburbs based on liveability
top_liveability_scores = liveability_scores.sort_values(ascending=False).head(7)

# Visualizing the liveability scores of these top 7 affordable suburbs
plt.figure(figsize=(10, 6))

# Creating a bar chart with suburbs on the x-axis and their respective liveability scores on the y-axis
bars = plt.bar(top_liveability_scores.index, top_liveability_scores.values, color='#66B2FF')

# Setting the title and labels for the graph
plt.title('Liveability Scores of Top 7 Affordable Suburbs')
plt.xlabel('Suburb')
plt.ylabel('Liveability Score')
plt.xticks(rotation=45) 

# Adding labels to each bar to show the exact liveability score value
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2.0, height / 2, f'{height:.1f}', ha='center', va='center', color='black')

# Displaying the plot with proper layout
plt.tight_layout()
plt.show()